# SOAP example

To install rascal:
+ mkdir ../build 
+ cd build
+ cmake -DCMAKE_BUILD_TYPE=Release -DBUILD_TESTS=ON ..
+ make -j 4
+ make install

TODO: Compare against an existing SOAP implementation in Python (e.g. quippy)

In [1]:
%env OMP_NUM_THREADS=1

env: OMP_NUM_THREADS=1


In [2]:
%matplotlib inline
from matplotlib import pyplot as plt

In [3]:
import rascal
import rascal.lib as lrl
import json

In [4]:
import ase
#from ase.io import read
from ase.visualize import view
import numpy as np
import sys

In [5]:
from rascal.representation.base import _representations, RepresentationFactory

In [6]:
_representations

{'coulomb': rascal.lib._rascal.RepresentationManager.SortedCoulomb_Strict_NeighbourList_Centers,
 'sphericalexpansion': rascal.lib._rascal.RepresentationManager.SphericalExpansion_Strict_NeighbourList_Centers}

In [7]:
test_hypers = {"interaction_cutoff": 6.0,
               "cutoff_smooth_width": 1.0, 
               "max_radial": 10,
               "max_angular": 8,
               "gaussian_sigma_type": "Constant",
               "gaussian_sigma_constant": 0.5}

In [8]:
#methane = rascal.utils.get_strict_neighbourlist(read(
#    '/local/scratch/mveit/data/phdproject/structures/methane-ccsdt-avtz-comp.xyz'),
#                                                cutoff=test_hypers['interaction_cutoff'])
methane = ase.io.read('/local/scratch/mveit/data/phdproject/structures/methane-ccsdt-avtz-comp.xyz')

In [9]:
json.dumps(test_hypers)

'{"interaction_cutoff": 6.0, "cutoff_smooth_width": 1.0, "max_radial": 10, "max_angular": 8, "gaussian_sigma_type": "Constant", "gaussian_sigma_constant": 0.5}'

In [10]:
sph_expn = rascal.representation.SphericalExpansion(**test_hypers)

In [11]:
sph_expn.hypers

{'interaction_cutoff': 6.0,
 'cutoff_smooth_width': 1.0,
 'max_radial': 10,
 'max_angular': 8,
 'gaussian_sigma_type': 'Constant',
 'gaussian_sigma_constant': 0.5,
 'n_species': 1}

In [12]:
expansions = sph_expn.transform([methane,])

In [16]:
dir(expansions)

['__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'append',
 'get_feature_matrix',
 'reserve',
 'shape',
 'size']

In [17]:
expansions.get_feature_matrix()

array([[ 1.45614444e-01,  8.34125492e-02,  8.34121180e-02,
         8.34118823e-02,  8.34129731e-02],
       [ 1.51520440e-01,  8.56712202e-02,  8.56707820e-02,
         8.56705343e-02,  8.56716713e-02],
       [ 1.15017638e-01,  6.47833515e-02,  6.47830269e-02,
         6.47828393e-02,  6.47836962e-02],
       ...,
       [ 1.33473428e-09,  2.00386882e-08, -1.36507936e-02,
        -9.58668851e-03,  2.32344582e-02],
       [ 4.83086450e-10,  2.04702850e-08, -1.19357179e-02,
        -8.38226947e-03,  2.03149496e-02],
       [ 1.13720106e-10,  1.24756641e-08, -6.45847379e-03,
        -4.53570756e-03,  1.09923510e-02]])

In [20]:
expansions.shape

(5, 810)

Good, should be 5 centres (5 atoms) * 10 radial * 81 ${}=(l+1)^2$

In [ ]:
# load a small subset of structures from QM9
frames = read(p+'tests/reference_data/dft-smiles_500.xyz',':')
cutoff = 3.

In [ ]:
# Compute the sorted coulomb matrices for the list of structures
#%timeit -n 20 -r 5 features = rep.transform(frames)

In [ ]:
# Compare with reference
#%timeit -n 20 -r 5 ref = get_coulomb_refs(frames,rep.size,cutoff,flavour='row-norm')
#np.allclose(test,ref)